In [136]:
import sqlite3
con = sqlite3.connect('db')

In [137]:
import pandas as pd

In [138]:
sqlite3.sqlite_version

'3.39.4'

In [139]:
df = pd.read_csv('database/german_credit_augmented.csv')
df.head()

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,NaN,NaN,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,52,male,2,own,quite rich,NaN,936,9,education,0,2008-04-27 08:23:07,200
3,35,female,3,own,little,NaN,1393,11,car,0,2007-05-06 10:58:22,45
4,28,male,2,own,little,NaN,776,12,radio/TV,0,2007-07-21 13:22:14,358


In [140]:
df.to_sql('german_credit', con, index=False, if_exists='replace')

1000

In [141]:
def select(sql):
    return pd.read_sql(sql, con)

In [142]:
sql = '''select t.sex,
count(1)
from german_credit t
group by t.sex'''

In [143]:
select(sql)

,sex,count(1)
0,female,310
1,male,690


In [144]:
df.groupby('sex')['credit_amount'].agg(['count', 'mean'])

,count,mean
sex,,
female,310,2877.774194
male,690,3448.040580


In [145]:
sql = '''select t.sex,
count(1) as cnt,
avg(t.credit_amount) as credit_amount_avg
from german_credit t
group by t.sex'''

In [146]:
select(sql)

,sex,cnt,credit_amount_avg
0,female,310,2877.774194
1,male,690,3448.040580


In [147]:
sql = '''select distinct t.housing from german_credit t'''

In [148]:
select(sql)

,housing
0,own
1,rent
2,free


In [149]:
sql = '''select count(t.checking_account), count(1) from german_credit t'''

In [150]:
select(sql)

,count(t.checking_account),count(1)
0,606,1000


In [151]:
sql = '''
select
t.checking_account,

count(*) as cnt,
avg(t.credit_amount) as credit_amount_avg

 from german_credit t
 group by t.checking_account
 '''

In [153]:
select(sql)

,checking_account,cnt,credit_amount_avg
0,None,394,3133.101523
1,little,274,3175.218978
2,moderate,269,3827.561338
3,rich,63,2177.650794


In [154]:
df.groupby('checking_account',dropna=False)['credit_amount'].count()

checking_account
little      274
moderate    269
rich         63
NaN         394
Name: credit_amount, dtype: int64

In [155]:
sql = '''select
sum(case when t.checking_account is null then 1 else 0 end) as is_null,

count(case when t.checking_account is null then 1 else null end) as is_null2

 from german_credit t'''

In [156]:
select(sql)

,is_null,is_null2
0,394,394


In [157]:
sql = '''select

t.checking_account,

coalesce(t.checking_account,'no_info')

 from german_credit t'''

In [158]:
select(sql)

,checking_account,"coalesce(t.checking_account,'no_info')"
0,None,no_info
1,little,little
2,None,no_info
3,None,no_info
4,None,no_info
...,...,...
995,little,little
996,moderate,moderate
997,moderate,moderate
998,None,no_info


In [159]:
t = pd.DataFrame({'id':[1,1,2],'name':['a','a','b']})

In [160]:
t

,id,name
0,1,a
1,1,a
2,2,b


In [161]:
t.to_sql('dupl_test',con,index=False,if_exists='replace')

3

In [162]:
sql = '''select * from dupl_test t'''

In [163]:
select(sql)

,id,name
0,1,a
1,1,a
2,2,b


In [164]:
sql = '''select t.id, t.name, count(1) as cnt from dupl_test t
group by t.id, t.name
'''

In [165]:
select(sql)

,id,name,cnt
0,1,a,2
1,2,b,1


In [166]:
sql = '''select t.id, t.name, count(1) as cnt from dupl_test t
group by t.id, t.name
having count(1) > 1
'''

In [167]:
select(sql)

,id,name,cnt
0,1,a,2


In [168]:
t = pd.DataFrame({'id':[1,1,2,2,3],
                  'name':['a','b','c','d','e']})

In [169]:
t

,id,name
0,1,a
1,1,b
2,2,c
3,2,d
4,3,e


In [170]:
t.to_sql('dupl_test',con,index=False,if_exists='replace')

5

In [171]:
sql = '''select * from dupl_test t'''

In [172]:
select(sql)

,id,name
0,1,a
1,1,b
2,2,c
3,2,d
4,3,e


In [173]:
sql = '''select t.id, count(1) as cnt from dupl_test t
group by t.id
having count(1) > 1
'''

In [174]:
select(sql)

,id,cnt
0,1,2
1,2,2


In [175]:
sql = '''select * from dupl_test t
where t.id in (1,2)'''

In [176]:
select(sql)

,id,name
0,1,a
1,1,b
2,2,c
3,2,d


In [177]:
sql = '''select t.id as cnt from dupl_test t
group by t.id
having count(1) > 1
'''

In [178]:
select(sql)

,cnt
0,1
1,2


In [179]:
sql = '''select * from dupl_test t
where t.id in (select t.id as cnt from dupl_test t
group by t.id
having count(1) > 1)'''

In [180]:
select(sql)

,id,name
0,1,a
1,1,b
2,2,c
3,2,d


In [181]:
'''Агрегация'''

'Агрегация'

In [182]:
sql = '''select date(t.contract_dt,'start of month') as month,
count(1) as credit_cnt,
count(distinct t.client_id) as client_id_unique,
sum(t.credit_amount) as credit_amount_sum,
avg(t.credit_amount) as credit_amount_avg

 from german_credit t
 group by date(t.contract_dt,'start of month')
 order by date(t.contract_dt,'start of month')
 '''

In [183]:
select(sql)

,month,credit_cnt,client_id_unique,credit_amount_sum,credit_amount_avg
0,2007-05-01,81,81,207663,2563.740741
1,2007-06-01,74,74,239594,3237.756757
2,2007-07-01,71,71,224333,3159.619718
3,2007-08-01,57,57,178569,3132.789474
4,2007-09-01,58,58,186909,3222.568966
5,2007-10-01,70,70,188534,2693.342857
6,2007-11-01,87,87,300504,3454.068966
7,2007-12-01,77,77,273973,3558.090909
8,2008-01-01,93,93,288080,3097.634409
9,2008-02-01,55,55,211128,3838.690909


In [184]:
"Интервалы (или бины или бакеты)"

'Интервалы (или бины или бакеты)'

In [185]:
sql = '''select t.credit_amount,

case when t.credit_amount < 1000 then '1. <1000'
when t.credit_amount < 2000 then '2. 1000-2000'
when t.credit_amount < 3000 then '3. 2000-3000'
when t.credit_amount >= 3000 then '4. >= 3000'

else 'other' end as credit_amount_bin

 from german_credit t'''

In [186]:
select(sql)

,credit_amount,credit_amount_bin
0,3074,4. >= 3000
1,1344,2. 1000-2000
2,936,1. <1000
3,1393,2. 1000-2000
4,776,1. <1000
...,...,...
995,2600,3. 2000-3000
996,4455,4. >= 3000
997,6403,4. >= 3000
998,5003,4. >= 3000


In [187]:
sql = '''select

case
when t.credit_amount < 1000 then '1. <1000'
when t.credit_amount < 2000 then '2. 1000-2000'
when t.credit_amount < 3000 then '3. 2000-3000'
when t.credit_amount >= 3000 then '4. >= 3000'

else 'other' end as credit_amount_bin,

count(1) as credit_cnt

 from german_credit t
 group by 1
 '''

In [188]:
select(sql)

,credit_amount_bin,credit_cnt
0,1. <1000,116
1,2. 1000-2000,316
2,3. 2000-3000,188
3,4. >= 3000,380


In [189]:
"Столбцы сводной таблицы"

'Столбцы сводной таблицы'

In [190]:
sql = '''select t.housing,

count(case when t.sex = 'female' then 1 else null end) as female,
count(case when t.sex = 'male' then 1 else null end) as male,

count(1) as cnt

from german_credit t
group  by t.housing'''

In [191]:
select(sql)

,housing,female,male,cnt
0,free,19,89,108
1,own,196,517,713
2,rent,95,84,179


In [192]:
sql = '''select distinct t.purpose from german_credit t'''

In [193]:
purpose = list(select(sql)['purpose'].values)

In [194]:
purpose

['radio/TV',
 'car',
 'education',
 'furniture/equipment',
 'repairs',
 'business',
 'domestic appliances',
 'vacation/others']

In [195]:
for p in purpose:
  print(f"count(case when t.purpose = '{p}' then 1 else null end) as {p.lower().replace(' ','').replace('/','')},")

count(case when t.purpose = 'radio/TV' then 1 else null end) as radiotv,
count(case when t.purpose = 'car' then 1 else null end) as car,
count(case when t.purpose = 'education' then 1 else null end) as education,
count(case when t.purpose = 'furniture/equipment' then 1 else null end) as furnitureequipment,
count(case when t.purpose = 'repairs' then 1 else null end) as repairs,
count(case when t.purpose = 'business' then 1 else null end) as business,
count(case when t.purpose = 'domestic appliances' then 1 else null end) as domesticappliances,
count(case when t.purpose = 'vacation/others' then 1 else null end) as vacationothers,


In [196]:
sql = '''select t.housing,

count(case when t.purpose = 'radio/TV' then 1 else null end) as radiotv,
count(case when t.purpose = 'car' then 1 else null end) as car,
count(case when t.purpose = 'education' then 1 else null end) as education,
count(case when t.purpose = 'furniture/equipment' then 1 else null end) as furnitureequipment,
count(case when t.purpose = 'repairs' then 1 else null end) as repairs,
count(case when t.purpose = 'business' then 1 else null end) as business,
count(case when t.purpose = 'domestic appliances' then 1 else null end) as domesticappliances,
count(case when t.purpose = 'vacation/others' then 1 else null end) as vacationothers,
count(1) as cnt

from german_credit t
group  by t.housing'''

In [197]:
select(sql)

,housing,radiotv,car,education,furnitureequipment,repairs,business,domesticappliances,vacationothers,cnt
0,free,15,55,15,11,3,5,0,4,108
1,own,227,219,34,122,17,76,10,8,713
2,rent,38,63,10,48,2,16,2,0,179
